# Analysis on user timelines
To see how these user-timelines were downloaded, see:

`sm_w2v.__main__.run_user_timeline_download()`

Data were downloaded to:

`data/user_timeline_tweets.json`

In [1]:
# load and concatenate tweet timeline-documents
import json

train = True

user_timelines = dict()
with open('../data/user_timeline_tweets.json') as f_in:
    for l in f_in:
        try:
            twt = json.loads(l)
        except:
            continue
        user_id_str = twt['user']['id_str']
        if user_id_str in user_timelines:
            user_timelines[user_id_str] = user_timelines[user_id_str] + ' ' + twt['text']
        else:
            user_timelines[user_id_str] = twt['text']

# clean/preprocess text

In [2]:
from gensim.parsing.preprocessing import STOPWORDS
from nltk.corpus import stopwords
import re


STOPWORDS = set(['rt', '', 'httpst', 'amp', 'https\'', 'ang',
                'ng', 'tcot', 'ako', 'sa', 'ko', 'rp', 'roc',
                'fe', 'sk', 'holi', 'asa', 'guru', 'yah', 'lista',
                 'hah' 'il', 'seo', 'casa', 'qotd', 'noh']).union(set(STOPWORDS)) \
    .union(set(stopwords.words('spanish'))) \
    .union(set(stopwords.words('portuguese'))) \
    .union(set(stopwords.words('german'))) \
    .union(set(stopwords.words('dutch')))

def clean_text(text):
    clean = re.sub(r'http.*$', '', text)
    clean = re.sub(r'[^a-zA-Z\s]', '', clean)

    clean = clean.lower()
    word_list = clean.split()
    word_list = [w for w in word_list if (w not in STOPWORDS)]

    return word_list

user_clean_timelines = dict()
for user_id_str in user_timelines.keys():
    user_clean_timelines[user_id_str] = clean_text(user_timelines[user_id_str])

# LDA topic modeling on timeline documents

In [3]:
from gensim.models.ldamodel import LdaModel
from gensim.models import TfidfModel
from gensim import corpora


user_timelines_keys = []
texts = []
for k, v in user_clean_timelines.items():
    user_timelines_keys.append(k)
    texts.append(v)

# filter out users with less than 200 words in the tweets in their timelines
long_timelines = [t for t in texts if len(t) > 200]
dictionary = corpora.Dictionary(long_timelines)
dictionary.filter_extremes(no_below=10, no_above=0.3, keep_n=100000)
corpus = [dictionary.doc2bow(text) for text in long_timelines]
tfidf = TfidfModel(corpus)
corpus = tfidf[corpus]
corpus = corpus[0:500]
print('number of user-timelines:', len(corpus))

if train:
    model = LdaModel(corpus, id2word=dictionary, num_topics=10, iterations=50, passes=10)
    model.save('../models/lda_user_timelines.model')

else:
    model = LdaModel.load('../models/lda_user_timelines.model')

number of user-timelines: 398


In [5]:
import pandas as pd
# idenitify topics of interest
pd.DataFrame([row[1].split(' + ') for row in model.print_topics(20)]).iloc[:,0:10]

0                  1                      2  \
0   0.001*wordpress     0.000*dentists               0.000*rb   
1   0.001*empirefox       0.001*crooks              0.001*csr   
2        0.001*vdeo          0.000*pra             0.000*gust   
3     0.002*hivaids       0.001*cdcstd           0.001*stigma   
4       0.001*nigga       0.001*niggas  0.001*realdonaldtrump   
5   0.001*fertility  0.001*infertility          0.001*armenia   
6   0.000*packaging       0.000*buenas            0.000*amigo   
7         0.001*mga           0.001*ka      0.001*socialmedia   
8     0.002*infosec      0.001*bigdata       0.000*multimedia   
9  0.001*nhsengland         0.001*sosa               0.000*tw   

                        3                   4                   5  \
0          0.000*aquarius       0.000*faction     0.000*reconnect   
1           0.000*payroll            0.000*ch         0.000*allin   
2           0.000*lgbtqia          0.000*rail  0.000*undetectable   
3  0.001*blacklivesmatter    0.001*realestate    0.001*getcovered   
4              0.001*lmao          0.001*cruz           0.001*gop   
5           0.000*collage           0.000*nig    0.000*deactivate   
6                0.000*bu  0.000*birthcontrol  0.000*reservations   
7              0.001*lang  0.001*entrepreneur         0.001*naman   
8               0.000*stl            0.000*fx       0.000*malware   
9          0.000*knitting     0.000*worldbank      0.000*thtorguk   

                  6                    7                 8  \
0  0.000*addictions           0.000*elxn      0.000*magnet   
1   0.000*capricorn     0.000*riptwitter       0.000*meded   
2          0.000*vc            0.000*ser         0.000*hoy   
3  0.001*infections  0.001*thebodydotcom         0.001*std   
4   0.000*gopdebate             0.000*ep         0.000*bae   
5  0.000*alzheimers      0.000*caregiver    0.000*dementia   
6          0.000*md        0.000*georgia  0.000*disponible   
7     0.001*bitcoin           0.001*yung       0.001*hindi   
8   0.000*precision            0.000*nes  0.000*freelancer   
9        0.000*weho        0.000*kathryn         0.000*nhs   

                     9  
0        0.000*sterile  
1       0.000*branding  
2            0.000*vai  
3            0.001*flu  
4        0.000*bitches  
5            0.000*ttc  
6  0.000*onceuponatime  
7      0.001*uniteblue  
8  0.000*cybersecurity  
9        0.000*crochet

In [6]:
# show users vs. features (topics)
features = dict()

for i, doc in enumerate(corpus):
    features[user_timelines_keys[i]] = model.get_document_topics(doc, minimum_probability=None)


In [7]:
# do time-plot of features for a specific user (topics) vs. time
import pandas as pd
import numpy as np

columns = ['topic0', 'topic1', 'topic2', 'topic3', 'topic4',
           'topic5', 'topic6', 'topic7', 'topic8', 'topic9']
index = user_timelines_keys
df = pd.DataFrame(data=np.zeros((len(index), len(columns))), columns=columns, index=index)

for doc_ind, user_doc in enumerate(features.values()):
    for topic in user_doc:
        df.iloc[doc_ind, topic[0]] = topic[1]

In [8]:
df.head()

topic0    topic1    topic2    topic3    topic4    topic5  \
329386706   0.000000  0.000000  0.000000  0.000000  0.912786  0.000000   
19591459    0.018266  0.018266  0.018266  0.121108  0.409216  0.341813   
403536803   0.017730  0.017730  0.017730  0.107228  0.500057  0.268605   
166772203   0.000000  0.000000  0.000000  0.313182  0.630885  0.000000   
1660996196  0.000000  0.000000  0.000000  0.125148  0.812491  0.000000   

              topic6    topic7    topic8    topic9  
329386706   0.000000  0.000000  0.000000  0.000000  
19591459    0.018266  0.018266  0.018266  0.018266  
403536803   0.017730  0.017731  0.017730  0.017730  
166772203   0.000000  0.000000  0.000000  0.000000  
1660996196  0.000000  0.000000  0.000000  0.000000

In [9]:
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

df.index.name = 'Users'
figure = sns.clustermap(df, yticklabels="")

figure.savefig('user_timeline_clustermap.pdf', format='pdf')